In [ ]:
import matplotlib.pyplot as plt
import keras
import numpy as np
import PIL
from matplotlib import image
from os import listdir
import os

from keras.models import Sequential
from keras import optimizers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [ ]:
# loading all of the images, resizing them,
# and storing them in a dictionary according to
# their respective classifications

loaded_images = {}

trick_classes = [
    '50-50 Grinds',
    'Crooks and Overcrooks',
]

for trick in trick_classes:
    img_num = 1
    image_directory = 'Skate Images/{}'.format(trick)
    
    for filename in listdir(image_directory):
        if '.DS_Store' in filename:
            continue
        if trick not in loaded_images.keys():
            loaded_images[trick] = []
        else:
            img_loc = 'Skate Images/{}/{}'.format(trick, filename)
            img_resized_loc = '{}/original-resized-{}.jpg'.format(image_directory, img_num)
            
            img = PIL.Image.open(img_loc)
            img_resized = img.resize((300,300))
            
            img_resized.save(img_resized_loc)            

            img_resized_data = image.imread(img_resized_loc)

            if img_resized_data.shape == (300,300):
                continue
            
            loaded_images[trick].append(img_resized_data)
            img_num += 1

In [ ]:
# Verifying the training images are resized to 256 x 256 pixels
for trick in loaded_images.keys():
    print(loaded_images[trick][0].shape)

In [ ]:
# Total number of training images
print(sum([len(trick_list) for trick_list in loaded_images.values()]))

In [ ]:
# Number of images per trick category
print("Number of images for each trick category: \n")
for trick in loaded_images.keys():
    num_images = len(loaded_images[trick])
    print('---> {} ({} images)'.format(trick, num_images))

In [ ]:
plt.imshow(loaded_images['Crooks and Overcrooks'][53])

In [ ]:
#Constructing ConvNet model
model = Sequential()
weight_initializer = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None)
regularizer = keras.regularizers.l2(l=0.1)
dense_regularizer = keras.regularizers.l2(l=0.1)

model.add(Conv2D(filters = 32, 
                 kernel_regularizer=regularizer,
                 kernel_initializer=weight_initializer,
                 kernel_size = (3,3),
                 input_shape = (300,300,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(filters = 32,
                 kernel_regularizer=regularizer,
                 kernel_initializer=weight_initializer,
                 kernel_size = (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(filters = 64,
                 kernel_regularizer=regularizer,
                 kernel_initializer=weight_initializer,
                 kernel_size = (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(filters = 64,
                 kernel_regularizer=regularizer,
                 kernel_initializer=weight_initializer,
                 kernel_size = (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())

model.add(Dense(128, kernel_initializer=weight_initializer,kernel_regularizer=dense_regularizer))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(128, kernel_initializer=weight_initializer,kernel_regularizer=dense_regularizer))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(2))
model.add(Activation('softmax'))

#Compiling model
sgd = optimizers.sgd(lr = 0.001)
adam = optimizers.adam(0.0001)
model.compile(optimizer = adam,
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
input_images = []
output = []

trick_map = {
    '5-0 Grinds' : '5-0 Grind',
    '50-50 Grinds' :  '50-50 Grind',
    'Crooks and Overcrooks' : 'Crooked Grind or Overcrook Grind',
    'Feeble Grinds' : 'Feeble Grind',
    'Lipslides and Boardslides' : 'Lipslide or Boardslide',
    'Nose Grinds' : 'Nose Grind',
    'Noseslides and Tailslides' : 'Noseslide or Tailslide',
    'Smith Grinds' : 'Smith Grind'
}

for trick in trick_classes:
    trick_images = loaded_images[trick]
    num_images = len(trick_images)
    image_labels = [trick_map[trick] for _ in range(num_images)]
    for image, label in zip(trick_images, image_labels):
        input_images.append(image)
        output.append(label)
        
output = np.array(output)

label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(output)
one_hot_labels = keras.utils.to_categorical(vec, num_classes = 2)

In [ ]:
input_images = np.array(input_images)
# input_images = np.array(list(map(lambda x: x / 255, input_images)))

In [ ]:
plt.imshow(input_images[68])
one_hot_labels[68]

In [ ]:
# define data preparation
datagen = ImageDataGenerator(height_shift_range = 0.15,
                             horizontal_flip=True,
                             width_shift_range = 0.15,
                             rotation_range = 15,
                             rescale= 1.0/255,
                             zoom_range=0.1,
                             shear_range=0.1)

# configure batch size and retrieve one batch of images
os.makedirs('images')

# fit parameters from data
datagen.fit(input_images)

i = 1
aug_images = []
aug_labels = []
for X_batch, y_batch in datagen.flow(input_images, one_hot_labels, batch_size=100, save_to_dir='images', save_prefix='aug', save_format='jpg'):
    if i == 50:
        break
    for X_aug_example, y_aug_example in zip(X_batch, y_batch):
        aug_images.append(X_aug_example)
        aug_labels.append(y_aug_example)
    i += 1

aug_images = np.array(aug_images)
aug_labels = np.array(aug_labels)

input_images = np.vstack((input_images, aug_images))
one_hot_labels = np.vstack((one_hot_labels, aug_labels))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(input_images, 
                                                     one_hot_labels)   

In [ ]:
print('We now have {} images in total.'.format(len(X_train) + len(X_test)))
print('There are {} training images and {} testing images.'.format(len(X_train), len(X_test)))

In [ ]:
plt.imshow(X_train[570])
print(y_train[570])

In [ ]:
model.fit(X_train,
          y_train,
          validation_data = (X_test, y_test),
          batch_size = 10,
          epochs = 10)

In [ ]:
keras.backend.clear_session()